In [1]:
from utils import prepare_jupyter
prepare_jupyter()

In [2]:
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
from structure import Dataset

rcParams['figure.figsize'] = 12, 10

# train_set, val_set = Dataset.create_cancer().split(0.5)
train_set, val_set = Dataset.create_iris().split(0.5)

/Users/tomek/.virtualenvs/masters/lib/python3.6/site-packages/lightgbm/__init__.py:47: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [ ]:
params = {
    'n_estimators': 3,
    'max_depth': 5
}

clf = xgb.XGBClassifier(**params)
clf.fit(train_set.X, train_set.y)

clf.predict(val_set.X)

In [ ]:
clf.n_estimators

In [ ]:
from structure.xgboost import XGBoostTree

# This is the best way to retrieve info about node (I can get additional data from `trees_to_dataframe`)
gv = xgb.to_graphviz(clf, num_trees=1)

xgb_tree = XGBoostTree.parse(str(gv), train_set)

xgb_preds = xgb_tree.predict(val_set.X)
xgb_preds

In [ ]:
plt.figure(figsize=(16, 10))
xgb.to_graphviz(clf).render()

In [ ]:
clf.get_booster().get_dump()[0]

In [ ]:
booster = clf.get_booster()

In [ ]:
booster.trees_to_dataframe().head(n=13)

In [ ]:
from math import ceil, log2

trees_df = booster.trees_to_dataframe()
first_tree = trees_df[trees_df['Tree'] == 0]
nodes = first_tree

def get_info(node):
    node_id = node[1]
    node_type = 'leaf' if node[3] == 'Leaf' else 'split'
    feature_idx = node[3][1:]
    value = node[8]
    cover = node[9]
    
    return {
        'id': node[1],
        'type': node_type,
        'feature_idx': int(feature_idx) if node_type == 'split' else None,
        'value': value,
        'count': int(node[9])
    }


def divide_nodes(nodes):
    n = len(nodes)
    sizes = map(lambda x: 2 ** x, range(0, int(ceil(log2(n)))))
    transformed = np.array([get_info(node) for node in nodes])
    
    divided_nodes = []
    tmp_nodes = transformed.copy()
    
    for size in sizes:
        nodes_slice = tmp_nodes[:size]
        tmp_nodes = tmp_nodes[size:]
        divided_nodes.append(nodes_slice)
        
    return divided_nodes
    
divided = divide_nodes(nodes.values)
divided

In [ ]:
leaf_str = 'leaf=-0.0697674453'
float(leaf_str.split('=')[1])

# XGBoost Ensemble testing

Wykorzystanie własnej struktury do ewaluacji

In [13]:
from structure.xgboost import XGBoostEnsemble

params = {
    'n_estimators': 3,
    'max_depth': 5
}

xgb_ensemble = XGBoostEnsemble(params)

In [22]:
iris_train, iris_val = Dataset.create_iris().split(0.90)
cancer_train, cancer_val = Dataset.create_cancer().split(0.90)

xgb_ensemble.fit(iris_train)

X_test = iris_val.X
y_test = iris_val.y

my_preds_raw = xgb_ensemble.predict_proba(X_test)
clf_preds_raw = xgb_ensemble.clf.predict_proba(X_test)

my_preds_cls = xgb_ensemble.predict(X_test)
clf_preds_cls = xgb_ensemble.predict(X_test)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,